<a href="https://colab.research.google.com/github/trizakiilu/Capstone-project-2022/blob/main/FINAL%20PRODUCT%20LAUNCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 17 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=e1a3b18ef78d89d0e0855a2f52da2d654ec881ef38cb35118f8dde37c0aade2b
  Stored in directory: /root/.cache/pip/wheels/b4/4b/8f/751e99d45f089bdf366a7d3e5066db3c2b84a62e4377f534d7
Successfully built face-recognition-models


In [2]:
from google.colab import drive
drive.mount('/content/drive/' , force_remount=True)
import face_recognition
import numpy as np
import cv2
import pandas as pd
import os
from google.colab.patches import cv2_imshow
import os
import PIL
import io
from base64 import b64decode, b64encode
import time

Mounted at /content/drive/


In [6]:
os.chdir('/content/drive/MyDrive/attendance')
import Video_Capture

In [7]:
a = Video_Capture
video_stream = a.Video_Capture().video_stream
video_frame = a.Video_Capture().video_frame
js_to_image = a.Video_Capture().js_to_image
bbox_to_bytes = a.Video_Capture().bbox_to_bytes

In [8]:
video_stream()
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    # convert JS response to OpenCV Image
    
    image_bytes = b64decode(js_reply["img"].split(',')[1])         
    #img = js_to_image(js_reply["img"])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # decode numpy array into OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)
    # create transparent overlay for bounding box
    bbox_array = np.zeros([600,600,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #face_location = face_recognition.face_locations(img)
    # get face region coordinates
    faces = face_cascade.detectMultiScale(gray)
    # get face bounding box for overlay
    for (x,y,w,h) in faces:
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    # format bbox into png for return
    bbox_PIL.save(iobuf, format='png')
    # format return string
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
      # update bbox so next frame gets new overlay
    bbox = bbox_bytes
    filename =  "/content/drive/MyDrive/attendance/test/4.png"
    cv2.imwrite(filename, img)

<IPython.core.display.Javascript object>

In [9]:
def read_train_images(folder , dim):
  X = []
  encode = []
  y = []
  Y = []
  n_samples=0
  none_image = 0
  for i in os.listdir(folder):
      #print(i) 
      img_file = cv2.imread( folder +'/' + i)
      #cv2_imshow(img_file)
      if img_file is not None:
          img_orginal=cv2.resize(img_file,dim)
          face_location = face_recognition.face_locations(img_orginal)
          encoded_img = face_recognition.face_encodings(img_orginal, face_location)
          #print(face_location)            
          X.append(img_orginal)
          encode.append(encoded_img)
          name = os.path.split(i)[-1]
          Y.append(name)
          n_samples+=1    
      else:
        none_image=+1
        print(f'there is {none_image} "not image" file in this folder')

  return X , Y, encode
dim = (600,600)
train_folder =  "/content/drive/MyDrive/attendance/train"
x , y , encoded_images= read_train_images(train_folder, dim)

In [10]:
def read_test_image(test_folder, dim, encoded_images, y):
  Attendees_list = []
  k=0
  for iter in os.listdir(test_folder):
    test_img = cv2.imread(test_folder + iter)
    j = 0
    for i in encoded_images:
      test_img=cv2.resize(test_img,dim)
      face_location_test = face_recognition.face_locations(test_img)
      encoded_test = face_recognition.face_encodings(test_img, face_location_test)
      
      if len(encoded_test) == 0:
        results = []
        
      else:
        results = face_recognition.compare_faces(np.array(i), np.array(encoded_test))
        print(iter)
        if results[0]:
          Attendees_list.append(y[j])
        else:
          pass
      j+=1
    k+=1   
  return Attendees_list, k
test_folder = "/content/drive/MyDrive/attendance/test/"
raw_list, K = read_test_image(test_folder, dim, encoded_images, y)


In [11]:
lis_of_attendees = []
[lis_of_attendees.append(x) for x in raw_list if x not in lis_of_attendees]

absent_list = list(set(y).difference(lis_of_attendees))
Absents = []
for i in absent_list:
  name = i.split('.')[0]
  Absents.append(name)

Attendees = []
for i in lis_of_attendees:
  name = i.split('.')[0]
  Attendees.append(name)
print(f'Attendees students are :{Attendees}')
print(f'Absents students are :{Absents}')

if K>len(raw_list):
  strange_no = K-len(raw_list)
  print(f'There are {strange_no} strange person in the class. Be careful!!')

Attendees students are :[]
Absents students are :['Ritvij', 'triza', 'Amin', 'Nasar']
There are 2 strange person in the class. Be careful!!


In [12]:
import pandas as pd
import time
from datetime import date
import csv

today = date.today().strftime('%Y-%m-%d %H:%M:%S')
os.chdir('/content/drive/MyDrive/attendance/')

In [13]:
!pip install xlsxwriter 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 24.9 MB/s 


In [14]:
import xlsxwriter
#worksheet = pd.read_excel('ClassList.xlsx')
workbook = xlsxwriter.Workbook('ClassList.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write('A1', 'Date')
worksheet.write('B1', 'Time of Check')
worksheet.write('C1', 'Amin')
worksheet.write('D1', 'Nasar')
worksheet.write('E1', 'Ritvij') 
worksheet.write('F1', 'Triza')

worksheet.write('A2', today.split(' ')[0])
worksheet.write('B2', today.split(' ')[1])
if 'Amin' in Attendees:
  worksheet.write('C2', 'Present')
else:
  worksheet.write('C2', 'Absent')
if 'Nasar' in Attendees:
  worksheet.write('D2', 'Present')
else:
  worksheet.write('D2', 'Absent')
if 'Ritvij' in Attendees:
  worksheet.write('E2', 'Present')
else:
  worksheet.write('E2', 'Absent')
if 'Triza' in Attendees:
  worksheet.write('F2', 'Present')
else:
  worksheet.write('F2', 'Absent')
workbook.close()